## データの前処理
データセットをファインチューニングのために整えていきます

#### 依存関係のインストール

In [ ]:
!pip install -r requirements.txt

#### 依存関係のインポート

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np

#### 定数宣言

In [ ]:
EMOTIONS = [
    "Joy",
    "Sadness",
    "Anticipation",
    "Surprise",
    "Anger",
    "Fear",
    "Disgust",
    "Trust",
]

PRETRAINED_MODEL_NAME = "line-corporation/line-distilbert-base-japanese"

#### データセットのロード
Pandas データフレームにロードします

In [2]:
df = pd.read_table("./dataset/wrime-ver2.tsv")

#### 訓練とテストのデータに分割する
カラム"Train/Dev/Test"というカラムを利用します

In [3]:
train_df = df[df["Train/Dev/Test"] == "train"].copy()
test_df = df[df["Train/Dev/Test"] == "test"].copy()

#### 感情カラムを統合する
投稿者の評価、3の客観評価の平均からなる合計16カラムを統合します、
主観と客観を用意されている1 : 3 の割合で統合したいので、客観評価に対して3を掛けます

In [4]:
def merge_emotions(splited_df):
    merged_df = splited_df.copy()
    emotions = EMOTIONS

    READER_NUM = 3
    for e in emotions:
        merged_df[f"Merged_{e}"] = (
            merged_df[f"Writer_{e}"] + merged_df[f"Avg. Readers_{e}"] * READER_NUM
        )
    merged_df["Merged_emotion_list"] = merged_df[
        [f"Merged_{e}" for e in emotions]
    ].values.tolist()

    return merged_df


train_merged_df = merge_emotions(train_df)
test_merged_df = merge_emotions(test_df)

#### トークナイザーのロードと保存

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    PRETRAINED_MODEL_NAME,
    clean_up_tokenization_spaces=False,
)
tokenizer.save_pretrained("./output/tokenizer")

('./output/tokenizer/tokenizer_config.json',
 './output/tokenizer/special_tokens_map.json',
 './output/tokenizer/spiece.model',
 './output/tokenizer/added_tokens.json')

#### トークナイズと正規化
Dataset にロードし、効率的にファインチューニングできるようにトークナイズとL1正規化を行います

In [ ]:
def format_dataset(batch, _tokenizer):
    formatted_batch = _tokenizer(
        batch["Sentence"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )
    formatted_batch["labels"] = [x / (np.sum(x) + 1e-8) for x in batch["Merged_emotion_list"]]

    return formatted_batch


target_columns = ["Sentence", "Merged_emotion_list"]
train_dataset = Dataset.from_pandas(train_merged_df[target_columns])
test_dataset = Dataset.from_pandas(test_merged_df[target_columns])

preprocessed_data = {
    "train": train_dataset.map(
        lambda batch: format_dataset(batch, tokenizer),
        batched=True,
        cache_file_name=None,
    ),
    "test": test_dataset.map(
        lambda batch: format_dataset(batch, tokenizer),
        batched=True,
        cache_file_name=None,
    ),
}

## ファインチューニング
前処理済みのデータセットを使って基盤モデルを訓練していきます

#### 基盤モデルのロード

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME,
    num_labels=len(EMOTIONS),
)

#### 訓練の設定

In [ ]:
training_args = TrainingArguments(
    output_dir="./output/result",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_data["train"],
    eval_dataset=preprocessed_data["test"],
)

#### 訓練開始とモデルの保存

In [ ]:
trainer.train()

model.save_pretrained("./output/model")